In [1]:
# load packages
import xarray as xr
import numpy as np
import pandas as pd

## Exploring projections

In [2]:
# define path to data
full_path = "/Users/kimbente/Desktop/Bayesian Optimisation/BO-ANT-project/raw-data/RACMO2.3p3/CON_data_version2/smb_monthlyS_ANT27_CONsettings_197901_201812.nc"

# open nc file as xarray dataset
ds = xr.open_dataset(full_path)
ds

<xarray.Dataset>
Dimensions:       (time: 480, bnds: 2, rlon: 262, rlat: 240, height: 1,
                   nblock1: 40, nblock2: 400)
Coordinates:
  * time          (time) datetime64[ns] 1979-01-16 ... 2018-12-16
  * rlon          (rlon) float64 -32.75 -32.5 -32.25 -32.0 ... 32.0 32.25 32.5
  * rlat          (rlat) float64 -30.0 -29.75 -29.5 -29.25 ... 29.25 29.5 29.75
    lon           (rlat, rlon) float64 ...
    lat           (rlat, rlon) float64 ...
  * height        (height) float64 0.0
Dimensions without coordinates: bnds, nblock1, nblock2
Data variables:
    time_bnds     (time, bnds) datetime64[ns] ...
    rotated_pole  float32 ...
    assigned      (time) float64 ...
    block1        (nblock1) float64 ...
    block2        (nblock2) float64 ...
    date_bnds     (time, bnds) float64 ...
    dir           (rlat, rlon) float64 ...
    dtg           (time) float64 ...
    hms_bnds      (time, bnds) float64 ...
    smb           (time, height, rlat, rlon) float64 ...
Attributes:
    CDI:           Climate Data Interface version 2.1.0 (https://mpimet.mpg.d...
    Conventions:   CF-1.4
    source:        RACMO2
    institution:   Royal Netherlands Meteorological Institute (KNMI)
    Domain:        ANT27y18
    Experiment:    ERA5_p3_cvdon_3hr_ws2_rssa100_zt025_rrfrez000025_rskin001
    CreationDate:  Mon Sep  7 10:59:51 2020
    comment:       asim2cdf: cpar=smb, iwmo=156, ilvt=105, ilev=5, idh=24, ih...
    title:          Surface Mass Balance ice sheet
    NCO:           netCDF Operators version 4.9.2 (Homepage = http://nco.sf.n...
    history:       Wed Sep 23 10:35:06 2020: ncrcat -O /scratch/ms/nl/nkcd/ex...
    CDO:           Climate Data Operators version 2.1.0 (https://mpimet.mpg.d...

In [3]:
ds.rotated_pole

<xarray.DataArray 'rotated_pole' ()>
[1 values with dtype=float32]
Attributes:
    grid_mapping_name:          rotated_latitude_longitude
    grid_north_pole_latitude:   -180.0
    grid_north_pole_longitude:  -170.0
    proj4_params:               -m 57.295779506 +proj=ob_tran +o_proj=latlon ...
    proj_parameters:            -m 57.295779506 +proj=ob_tran +o_proj=latlon ...
    projection_name:            rotated_latitude_longitude
    long_name:                  projection details
    EPSG_code:

RACMO data is provided on a rotated pole projection. 

Different CORDEX domains have different rotations.

proj4_params :
-m 57.295779506 +proj=ob_tran +o_proj=latlon +o_lat_p=-180.0 +lon_0=10.0

- cartopy.crs.CRS is the class for coordinate reference systems (crs)
  - parameters: proj4_params
    - Defined by defining the new pole
    - '+proj=ob_tran' (general) oblique transformation
    - '+o_proj' projection to be used (order matters)
    - '+o_lat_p' Latitude of the North pole of the unrotated source CRS, expressed in the rotated geographic CRS.
    - '+o_lon_p' Longitude of the North pole of the unrotated source CRS, expressed in the rotated geographic CRS.
    - ['+lon_0' Longitude of projection center.] Default: 0.0 The default convention is to interpret this value as decimal degrees.

[Source of proj details](https://proj.org/en/9.2/operations/projections/ob_tran.html)
[Tool to check projections](https://agrimetsoft.com/Cordex%20Coordinate%20Rotation)
[Notebook by Stef Lhermitte](https://gitlab.tudelft.nl/slhermitte/manuals/-/blob/master/RACMO_reproject.md)
rotated_latitude_longitude

RotPole (13.08; -6.08) (lon, lat)
- The new [rlat, rlon] == [0.0, 0.0] is at [-90.0,	-170.0] which is [90S, 170W]

## Other projections

[WGS 84 / Antarctic Polar Stereographic](https://epsg.io/3031)

    # PROJECTION
    # define projection from rotated pole to APS
    # always x y or lon lat order
    rotatedpole_to_APS71_func = pyproj.Transformer.from_crs(crs_from = pyproj.CRS("-m 57.295779506 +proj=ob_tran +o_proj=latlon +o_lat_p=-180.0 +lon_0=10.0"),  
                                                  crs_to = pyproj.CRS("+proj=stere +lat_0=-90 +lat_ts=-71 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +type=crs"), 
                                                  always_xy = True)

        # create meshgrid from rotated_pole grid
    rlon_2d, rlat_2d = np.meshgrid(dataset_smb_xr.rlon.values, dataset_smb_xr.rlat.values)
    # transform points
    x1_2d, x2_2d = rotatedpole_to_APS71_func.transform(rlon_2d, rlat_2d)

In [16]:
lon_df = xr.DataArray.to_dataframe(ds.coords["lon"], name = "lon")
lon_df[lon_df["lat"] < -89.]

lon        lat         lon
rlat  rlon                                    
-0.75 -0.50 -136.311775 -89.098620 -136.311775
      -0.25 -151.566078 -89.209433 -151.566078
       0.00 -170.000000 -89.250000 -170.000000
       0.25  171.566078 -89.209433  171.566078
       0.50  156.311775 -89.098620  156.311775
-0.50 -0.75 -113.691498 -89.098620 -113.691498
      -0.50 -125.001091 -89.292897 -125.001091
      -0.25 -143.435594 -89.440984 -143.435594
       0.00 -170.000000 -89.500000 -170.000000
       0.25  163.435594 -89.440984  163.435594
       0.50  145.001091 -89.292897  145.001091
       0.75  133.691498 -89.098620  133.691498
-0.25 -0.75  -98.435558 -89.209433  -98.435558
      -0.50 -106.565497 -89.440984 -106.565497
      -0.25 -125.000273 -89.646447 -125.000273
       0.00 -170.000000 -89.750000 -170.000000
       0.25  145.000273 -89.646447  145.000273
       0.50  126.565497 -89.440984  126.565497
       0.75  118.435558 -89.209433  118.435558
 0.00 -0.75  -80.000016 -89.250000  -80.000016
      -0.50  -80.000024 -89.500000  -80.000024
      -0.25  -80.000047 -89.750000  -80.000047
       0.00 -170.000000 -90.000000 -170.000000
       0.25  100.000047 -89.750000  100.000047
       0.50  100.000024 -89.500000  100.000024
       0.75  100.000016 -89.250000  100.000016
 0.25 -0.75  -61.564470 -89.209433  -61.564470
      -0.50  -53.434541 -89.440984  -53.434541
      -0.25  -34.999774 -89.646447  -34.999774
       0.00    9.999965 -89.750000    9.999965
       0.25   54.999774 -89.646447   54.999774
       0.50   73.434541 -89.440984   73.434541
       0.75   81.564470 -89.209433   81.564470
 0.50 -0.75  -46.308524 -89.098620  -46.308524
      -0.50  -34.998933 -89.292898  -34.998933
      -0.25  -16.564425 -89.440984  -16.564425
       0.00   10.000000 -89.500000   10.000000
       0.25   36.564425 -89.440984   36.564425
       0.50   54.998933 -89.292898   54.998933
       0.75   66.308524 -89.098620   66.308524
 0.75 -0.50  -23.688240 -89.098620  -23.688240
      -0.25   -8.433931 -89.209433   -8.433931
       0.00   10.000000 -89.250000   10.000000
       0.25   28.433931 -89.209433   28.433931
       0.50   43.688240 -89.098620   43.688240
 1.00  0.00   10.000000 -89.000000   10.000000

# Check McMurdo

GEO URI  
latitude: -77.8  
longitude: 166.6

In [34]:
lon_df.iloc[:, :-1]

lon        lat
rlat   rlon                         
-30.00 -32.75 -126.863005 -46.749179
       -32.50 -127.057818 -46.919805
       -32.25 -127.254420 -47.089806
       -32.00 -127.452823 -47.259174
       -31.75 -127.653042 -47.427902
...                   ...        ...
 29.75  31.50   52.433021 -47.753669
        31.75   52.635256 -47.584661
        32.00   52.835646 -47.415007
        32.25   53.034203 -47.244715
        32.50   53.230944 -47.073792

[62880 rows x 2 columns]

In [35]:
lon_df = xr.DataArray.to_dataframe(ds.coords["lon"], name = "lon")
lon_df = lon_df.iloc[:, :-1]
lon_df[(lon_df["lat"] < -76.0) & (lon_df["lat"] > -79.0) & (lon_df["lon"] < 168.0) & (lon_df["lon"] > 165.0)]
#lon_df[(lon_df["lat"] < -76.0) & (lon_df["lat"] > -79.0)]
# lon_df[(lon_df["lon"] < 168.0) & (lon_df["lon"] > 165.0)]

lon        lat
rlat   rlon                       
-12.75 5.25  167.982627 -76.227989
       5.50  167.043588 -76.132368
       5.75  166.117793 -76.033000
-12.50 5.25  167.572242 -76.458450
       5.50  166.619614 -76.361149
       5.75  165.680878 -76.260061
-12.25 5.25  167.147938 -76.688219
       5.50  166.181477 -76.589188
       5.75  165.229593 -76.486327
-12.00 5.00  167.704628 -77.014115
       5.25  166.709027 -76.917260
       5.50  165.728490 -76.816445
-11.75 5.00  167.265468 -77.244183
       5.25  166.254778 -77.145533
       5.50  165.259928 -77.042879
-11.50 4.75  167.852922 -77.569852
       5.00  166.810474 -77.473499
       5.25  165.784414 -77.372998
-11.25 4.75  167.397795 -77.800239
       5.00  166.338817 -77.702023
       5.25  165.297111 -77.599609
-11.00 4.75  166.925447 -78.029850
       5.00  165.849612 -77.929708
-10.75 4.50  167.546847 -78.356365
       4.75  166.434932 -78.258641
       5.00  165.341918 -78.156507
-10.50 4.50  167.055728 -78.586293
       4.75  165.925237 -78.486562
-10.25 4.25  167.714939 -78.912537
       4.50  166.544726 -78.815371
       4.75  165.395280 -78.713562